# Lesson 5: Filesystem modification

**Objectives**: learn about OFRAK's filesystem unpacking and modification capabilities; modify a binary from within a filesystem and the filesystem itself

In this section, our boring old "Hello, World!" binary will be embedded within a SquashFS filesystem image, with the path `src/hello_world`:

```
squashfs_dir
└── src
    └── hello_world
```

Let's first create that image:

In [6]:
from ofrak_tutorial.helper_functions import HELLO_WORLD_SOURCE, create_binary

!rm -rf squashfs_dir && mkdir -p squashfs_dir/src

create_binary(HELLO_WORLD_SOURCE, "squashfs_dir/src/hello_world")

!rm -f image.sqsh && mksquashfs squashfs_dir image.sqsh >/dev/null

Now we'll want to do a few things to this image:
- modify the binary to output something even more fun and furry than in [Lesson 1](1_simple_string_modification.ipynb) 😼😼
- modify the permission bits and extended attributes of the binary

First, let's see how to unpack and view the current SquashFS image:

In [7]:
from ofrak import OFRAK

ofrak = OFRAK()
basic_context = await ofrak.create_ofrak_context()
root_resource = await basic_context.create_root_resource_from_file("image.sqsh")
unpack_result = await root_resource.unpack_recursively()

Which tags does our unpacked resource have?

In [8]:
for tag in sorted(root_resource.get_tags(), key=str):
    print(tag)

File
FilesystemEntry
FilesystemRoot
GenericBinary
SquashfsFilesystem


`SquashfsFilesystem` looks like the most interesting tag for the operations we want to perform. Let's create a resource view of that type:

In [9]:
from ofrak.core import SquashfsFilesystem

squashfs_view = await root_resource.view_as(SquashfsFilesystem)

... And let's get the hello world binary from within it.

In [10]:
hello_world_program = await squashfs_view.get_entry("src/hello_world")

Okay. What are the current permission bits and extended attributes?

In [11]:
print(f"current permission bits: {hello_world_program.stat.st_mode:o}")

current permission bits: 100755
current extended attributes: {}


We observe that our binary is possibly even more boring in its permissions and extended attributes than in what it outputs. Let's remedy all of this, in one go:

In [12]:
import stat

from ofrak.core import BinaryPatchConfig, BinaryPatchModifier

async def modify_squashfsed_program(ofrak_context, input_filename, output_filename):
    root_resource = await ofrak_context.create_root_resource_from_file(input_filename)

    await root_resource.unpack_recursively()
    squashfs_view = await root_resource.view_as(SquashfsFilesystem)
    hello_world_program = await squashfs_view.get_entry("src/hello_world")

    # Get the "Hello, World!" string location in the program and patch it with "More meow!"
    program_data = await hello_world_program.resource.get_data()
    hello_world_offset = program_data.find(b"Hello, World!")

    await hello_world_program.resource.run(
        BinaryPatchModifier,
        BinaryPatchConfig(hello_world_offset, b"More meow!\0")
    )

    # Modify the program permission bits before repacking:
    # Misanthropic and eyebrow-raising permission bits...
    await hello_world_program.modify_stat_attribute(stat.ST_MODE, 0o100300)

    await root_resource.pack()
    await root_resource.flush_to_disk(output_filename)

Does it work?

In [13]:
await modify_squashfsed_program(basic_context, "image.sqsh", "modified_image.sqsh")

[      packer.py:   71] The SquashfsPacker packer has already been run on resource 8269ef5a26634f949406de22b9989340


In [14]:
%%bash

rm -rf modified_squashfs_dir && unsquashfs -d modified_squashfs_dir modified_image.sqsh >/dev/null
./modified_squashfs_dir/src/hello_world

More meow!


In [15]:
import os

print(f"new permission bits: {os.stat('modified_squashfs_dir/src/hello_world').st_mode:o}")

new permission bits: 100300


[Next page](6_code_insertion_with_extension.ipynb)